<a href="https://colab.research.google.com/github/junxnone/examples/blob/master/algo/faiss_basic.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [4]:
!pip3 install faiss numpy

In [9]:
!apt install libomp-dev

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following additional packages will be installed:
  libomp5
Suggested packages:
  libomp-doc
The following NEW packages will be installed:
  libomp-dev libomp5
0 upgraded, 2 newly installed, 0 to remove and 28 not upgraded.
Need to get 239 kB of archives.
After this operation, 804 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp5 amd64 5.0.1-1 [234 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 libomp-dev amd64 5.0.1-1 [5,088 B]
Fetched 239 kB in 1s (326 kB/s)
Selecting previously unselected package libomp5:amd64.
(Reading database ... 132681 files and directories currently installed.)
Preparing to unpack .../libomp5_5.0.1-1_amd64.deb ...
Unpacking libomp5:amd64 (5.0.1-1) ...
Selecting previously unselected package libomp-dev.
Preparing to unpack .../libomp-dev_5.0.1-1_amd64.deb ...
Unpacking libomp-dev (5.0.1-

In [0]:
import faiss
import numpy as np

In [23]:
d = 1024                           # dimension
nb = 64                      # database size
np.random.seed(1234)             # make reproducible
xb = np.random.random((nb, d)).astype('float32')
# xb[:, 0] += np.arange(nb) / 1000.
print(xb.shape)

(64, 1024)


# FlatL2

In [15]:
index = faiss.IndexFlatL2(d)   # build the index
index.add(xb)                  # add vectors to the index
print(index.ntotal)

64


In [24]:
t1=np.random.random((1, d)).astype('float32')
print(t1.shape)
D, I = index.search(t1, 5)     # actual search
print(I)
print(D)

(1, 1024)
[[48 40 29 55 54]]
[[157.486   159.61536 159.80894 163.25558 164.56345]]


# Kmeans

In [25]:
ncentroids = 7
niter = 200
verbose = True
d = xb.shape[1]
kmeans = faiss.Kmeans(d, ncentroids, niter=niter, verbose=verbose)
kmeans.train(xb)

4813.1504

In [26]:
D, I = kmeans.index.search(xb, 1)
print(I.T)
print(D.T)

[[1 3 5 4 3 5 6 4 2 2 6 0 3 3 6 4 6 4 6 2 4 0 6 6 6 3 4 3 2 6 2 3 6 6 5 3
  4 4 3 6 6 4 3 5 4 3 2 2 1 2 6 5 6 6 3 2 2 4 2 3 4 2 5 3]]
[[39.16687  81.2424   71.83099  74.82605  80.40521  66.74335  77.26056
  78.24255  79.058105 76.436646 79.79193  39.23761  82.380615 80.03906
  78.1051   72.03485  78.605774 75.52905  87.35718  77.765625 79.619385
  39.237793 80.89673  78.6095   77.430176 73.90021  73.64612  83.806854
  80.81787  76.93744  71.69946  76.954895 82.90747  74.43231  71.99042
  80.73645  81.57654  75.703735 72.405396 81.13452  79.72034  74.12305
  81.19641  68.994934 82.51813  81.76337  78.07623  76.30304  39.166687
  79.2229   77.878296 69.590454 77.61139  79.16815  80.833496 77.67084
  76.64105  76.6427   79.579834 74.54474  80.09155  79.50922  72.045715
  79.75543 ]]


In [27]:
index = faiss.IndexFlatL2(d)
index.add(xb)
D, I = index.search (kmeans.centroids, 1)
print(kmeans.centroids.shape)
print(I)

(7, 1024)
[[11]
 [ 0]
 [30]
 [38]
 [15]
 [ 5]
 [33]]
